In [53]:
import spacy
import fasttext
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import nltk
import re,string
nltk.download('stopwords')
from nltk.corpus import stopwords
stopword=set(stopwords.words('english'))
stemmer = nltk.SnowballStemmer("english")

#read the dataset with name "Fake_Real_Data.csv" and store it in a variable df
df = pd.read_csv("C:\\Users\\Admin\\Downloads\\twitter_data.csv")

#print the shape of dataframe
print(df.shape)

#print top 5 rows
df.head(5)

(24783, 7)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [2]:
nlp = spacy.load("en_core_web_lg")

In [54]:
df["labels"] = df['class'].map({0: "Hate_Speech", 1: "Offensive_Speech", 2: "No_Hate_and_Offensive_Speech"})
df = df[["tweet", "labels"]]

In [38]:
#df["labels"] = df['class'].map({1: "Hate_Speech", 0: "No_Hate_and_Offensive_Speech"})
#df = df[["tweet", "labels"]]

In [55]:
def preprocess(text):
        text = str(text)
        test_list = text.split()
        text = ""
        for word in test_list:
            if word.endswith("ing"):
                word=word[:-3]
            text+=word
            text+=" "
        doc = nlp(text)
        filtered_tokens = []
        
        for token in doc:
            if token.is_stop or token.is_punct:
                continue
            filtered_tokens.append(token.lemma_)
            
        return " ".join(filtered_tokens)

In [56]:
df["tweet_new"] = df.tweet.apply(preprocess)

In [57]:
df.head()

,tweet,labels,tweet_new
0,!!! RT @mayasolovely: As a woman you shouldn't...,No_Hate_and_Offensive_Speech,RT @mayasolovely woman complain clean house am...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive_Speech,RT @mleew17 boy dats cold tyga dwn bad cuffin ...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive_Speech,RT @urkindofbrand Dawg RT @80sbaby4life fuck b...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive_Speech,RT @C_G_Anderson @viva_based look like tranny
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive_Speech,RT @ShenikaRoberts shit hear true faker bitch ...


In [58]:
def clean(text):
    text = str (text). lower()
    text = re. sub('[.?]', '', text)
    text = re. sub('https?://\S+|www.\S+', '', text)
    text = re. sub('<.?>+', '', text)
    text = re. sub('[%s]' % re. escape(string. punctuation), '', text)
    text = re. sub('\n', '', text)
    text = re. sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ". join(text)
    text = [stemmer. stem(word) for word in text. split(' ')]
    text=" ". join(text)
    return text

In [59]:
df["tweet_new"] = df.tweet.apply(clean)

In [60]:
df['labels'] = '__label__' + df['labels'].astype(str)
df.head(5)

,tweet,labels,tweet_new
0,!!! RT @mayasolovely: As a woman you shouldn't...,__label__No_Hate_and_Offensive_Speech,rt mayasolov woman shouldnt complain clean ho...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,__label__Offensive_Speech,rt mlee boy dat coldtyga dwn bad cuffin dat h...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,__label__Offensive_Speech,rt urkindofbrand dawg rt babif ever fuck bitc...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,__label__Offensive_Speech,rt cganderson vivabas look like tranni
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,__label__Offensive_Speech,rt shenikarobert shit hear might true might f...


In [61]:
df['category_description'] = df['labels'] + ' ' + df['tweet_new']
df.head(3)

,tweet,labels,tweet_new,category_description
0,!!! RT @mayasolovely: As a woman you shouldn't...,__label__No_Hate_and_Offensive_Speech,rt mayasolov woman shouldnt complain clean ho...,__label__No_Hate_and_Offensive_Speech rt maya...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,__label__Offensive_Speech,rt mlee boy dat coldtyga dwn bad cuffin dat h...,__label__Offensive_Speech rt mlee boy dat col...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,__label__Offensive_Speech,rt urkindofbrand dawg rt babif ever fuck bitc...,__label__Offensive_Speech rt urkindofbrand da...


In [62]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

In [63]:
train.shape, test.shape

((19826, 4), (4957, 4))

In [64]:
train.to_csv("hate.train", columns=["category_description"], index=False, header=False)
test.to_csv("hate.test", columns=["category_description"], index=False, header=False)

In [65]:
model = fasttext.train_supervised(input="hate.train")
model.test("hate.test")

(4957, 0.8997377446035909, 0.8997377446035909)

In [66]:
txt = "fuck"
txt = preprocess(txt)
model.predict(txt)

(('__label__Offensive_Speech',), array([0.99730641]))